In [1]:
import cartolafc
import pandas as pd
from difflib import get_close_matches
import json, re
from pathlib import Path
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', 50)            # permite a visualização de 50 colunas do dataframe
pd.options.display.float_format = '{:.2f}'.format   # pandas: para todos os números aparecerem com duas casas decimais

# Cria uma instância da API
api = cartolafc.Api(attempts=5)

2025-08-16 18:39:24,672 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [2]:
# Carregar o arquivo CSV
df_times = pd.read_excel("times.xlsx")

# Ver os dados carregados
display(df_times.head())

,Nome
0,A Lenda Super Vascão f.c
1,A Lenda Super Vasco F.c
2,Analove10 ITAQUI GRANDE!!
3,BordonFC
4,BORGES ITAQUI F.C.


In [3]:
# 👉 ajuste o caminho do HTML salvo
HTML_PATH = Path("pagina_liga_seria_A.html")

# carrega html
html = HTML_PATH.read_text(encoding="utf-8", errors="ignore")
soup = BeautifulSoup(html, "html.parser")

# 1) seletor principal dessa página
nomes = [el.get_text(strip=True) for el in soup.select("p.card-participantes-pontos-corridos__time")]

# 2) fallbacks (caso salve outra variante da página)
if not nomes:
    nomes = [el.get_text(strip=True) for el in soup.select("p.nome-time")]
if not nomes:
    nomes = [(a.get("title") or a.get_text(strip=True)).strip()
             for a in soup.select("a.link-perfil-time") if (a.get("title") or a.get_text(strip=True))]

# 3) fallback final via regex (se o HTML veio “achatado”)
if not nomes:
    for m in re.finditer(r'class="card-participantes-pontos-corridos__time"\s*>\s*([^<]+)<', html, re.I):
        nomes.append(m.group(1).strip())

# dedup + ordena alfabética
visto, times = set(), []
for n in nomes:
    n = re.sub(r"\s+", " ", n).strip()
    if n and n not in visto:
        visto.add(n); times.append(n)
times.sort(key=str.casefold)

print(f"{len(times)} times encontrados")
print(times)

# snippet JS ao lado do HTML (se quiser usar no site)
OUT_JS = HTML_PATH.with_name("participantesLiga_serie_A.js")
OUT_JS.write_text("window.participantesLiga = " + json.dumps(times, ensure_ascii=False, indent=2) + ";\n",
                  encoding="utf-8")
print(f"Snippet salvo em: {OUT_JS}")


20 times encontrados
['A Lenda Super Vasco F.c', 'Analove10 ITAQUI GRANDE!!', 'BORGES ITAQUI F.C.', 'cartola scheuer', 'DM Studio', 'Dom Camillo68', 'FBC Colorado', 'Fedato Futebol Clube', 'Gremiomaniasm', 'lsauer fc', 'MauHumor F.C.', 'pura bucha /botafogo', 'RHANKA DENTY FC25', 'Rolo Compressor ZN', 'seralex', 'Tatols Beants F.C', 'TEAM LOPES 99', 'Texas Club 2025', 'TIGRE LEON', 'Time do S.A.P.O']
Snippet salvo em: participantesLiga_serie_A.js


In [4]:
JS_PATH = Path("participantesLiga_serie_A.js")  # ajuste o caminho se estiver em outra pasta

# extrai o array do snippet: window.participantesLiga = [ ... ];
m = re.search(r'window\.participantesLiga\s*=\s*(\[[\s\S]*?\])\s*;', JS_PATH.read_text(encoding="utf-8"))
if not m:
    raise ValueError(f"Não consegui achar a lista dentro de {JS_PATH}")

nomes_times = json.loads(m.group(1))

# (opcional) dedup + ordena
nomes_times = sorted(dict.fromkeys(nomes_times), key=str.casefold)

print(len(nomes_times), "times")
nomes_times

20 times


['A Lenda Super Vasco F.c',
 'Analove10 ITAQUI GRANDE!!',
 'BORGES ITAQUI F.C.',
 'cartola scheuer',
 'DM Studio',
 'Dom Camillo68',
 'FBC Colorado',
 'Fedato Futebol Clube',
 'Gremiomaniasm',
 'lsauer fc',
 'MauHumor F.C.',
 'pura bucha /botafogo',
 'RHANKA DENTY FC25',
 'Rolo Compressor ZN',
 'seralex',
 'Tatols Beants F.C',
 'TEAM LOPES 99',
 'Texas Club 2025',
 'TIGRE LEON',
 'Time do S.A.P.O']

### Buscar IDs dos times no Cartola

In [5]:
# Dicionário para armazenar os IDs dos times
ids_times = {}

# Função robusta para buscar ID do time por nome
from difflib import get_close_matches

def buscar_id_time(nome_time):
    try:
        times = api.times(query=nome_time)

        # 🛡️ Proteção caso a resposta seja uma string (ex: erro HTML ou mensagem)
        if not isinstance(times, list):
            print(f"⚠️ Resposta inesperada para '{nome_time}':", times)
            return None

        nomes_api = [time.nome for time in times]

        # Comparação flexível (removendo acentos, pontos, etc)
        nome_base = nome_time.lower().replace(".", "").replace("fc", "").replace("f.c", "").strip()
        nomes_api_base = [n.lower().replace(".", "").replace("fc", "").replace("f.c", "").strip() for n in nomes_api]

        nome_proximo = get_close_matches(nome_base, nomes_api_base, n=1, cutoff=0.6)

        if nome_proximo:
            idx = nomes_api_base.index(nome_proximo[0])
            return times[idx].id

    except Exception as e:
        print(f"❌ Erro ao buscar ID para o time '{nome_time}': {e}")
    return None


# Buscar IDs automaticamente, sem sobrescrever os que já estiverem no dicionário
nao_encontrados = []

for nome in nomes_times:
    if nome not in ids_times:  # Protege os manuais
        time_id = buscar_id_time(nome)
        if time_id:
            ids_times[nome] = time_id
        else:
            nao_encontrados.append(nome)

# Mostrar resultado
print("IDs encontrados:", len(ids_times))
print("Times não encontrados automaticamente:", nao_encontrados)

IDs encontrados: 20
Times não encontrados automaticamente: []


### Gerar DataFrame com nome do time, ID e URL

In [6]:
# Gerar DataFrame com nome do time, ID e URL
df_urls = pd.DataFrame([
    {"Nome do Time": nome, "ID do Time": time_id, "Link do Time": f"https://cartola.globo.com/#!/time/{time_id}"}
    for nome, time_id in ids_times.items()
])

# Caminho e nome do arquivo Excel
caminho_excel = "links_times_cartola_liga_serie_A.xlsx"

# Salvar o DataFrame em Excel
df_urls.to_excel(caminho_excel, index=False)

print(f"✅ Arquivo salvo com sucesso: {caminho_excel}")


# Exibir como tabela
display(df_urls.head(30))

✅ Arquivo salvo com sucesso: links_times_cartola_liga_serie_A.xlsx


,Nome do Time,ID do Time,Link do Time
0,A Lenda Super Vasco F.c,117598,https://cartola.globo.com/#!/time/117598
1,Analove10 ITAQUI GRANDE!!,44574236,https://cartola.globo.com/#!/time/44574236
2,BORGES ITAQUI F.C.,3914981,https://cartola.globo.com/#!/time/3914981
3,cartola scheuer,3851966,https://cartola.globo.com/#!/time/3851966
4,DM Studio,387186,https://cartola.globo.com/#!/time/387186
5,Dom Camillo68,20696550,https://cartola.globo.com/#!/time/20696550
6,FBC Colorado,186283,https://cartola.globo.com/#!/time/186283
7,Fedato Futebol Clube,18642587,https://cartola.globo.com/#!/time/18642587
8,Gremiomaniasm,528730,https://cartola.globo.com/#!/time/528730
9,lsauer fc,44810918,https://cartola.globo.com/#!/time/44810918


### Gerar o dicionário ID -> Nome do Time

In [7]:
# Gerar o dicionário ID -> Nome do Time
nomes_por_id = dict(zip(df_urls["ID do Time"], df_urls["Nome do Time"]))

# Mostrar parte do dicionário
dict(list(nomes_por_id.items())[:5])

{117598: 'A Lenda Super Vasco F.c',
 44574236: 'Analove10 ITAQUI GRANDE!!',
 3914981: 'BORGES ITAQUI F.C.',
 3851966: 'cartola scheuer',
 387186: 'DM Studio'}

In [8]:
TURNO_INICIO = 20
TOTAL_RODADAS = 19

# Tente usar a exceção específica; senão, caia para Exception
try:
    from cartolafc import CartolaFCError as _CartolaErr
except Exception:
    _CartolaErr = Exception

def campeonato_comecou(api, ids_times):
    """
    Considera que o 2º turno começou se:
      - o mercado indica rodada_atual >= 20 E
      - há pelo menos uma pontuação não-nula na rodada 20 para algum time.
    """
    try:
        rodada_atual = getattr(api.mercado(), "rodada_atual", None)
    except _CartolaErr:
        rodada_atual = None

    if rodada_atual is None or rodada_atual < TURNO_INICIO:
        return False

    for time_id in ids_times.values():
        try:
            p = api.time(time_id=time_id, rodada=TURNO_INICIO).ultima_pontuacao
            if p is not None:
                return True
        except _CartolaErr:
            continue
    return False

def obter_pontuacao_por_rodada(api, time_id, rodada_atual):
    """
    Busca apenas rodadas concluídas: [20, rodada_atual-1].
    """
    pontuacoes = {}
    # fim exclusivo; para parar em rodada_atual-1 use range(..., rodada_atual)
    fim = min(max(rodada_atual, TURNO_INICIO), TOTAL_RODADAS + 1)
    for rodada in range(TURNO_INICIO, fim):
        try:
            time_r = api.time(time_id=time_id, rodada=rodada)
            pontuacoes[rodada] = time_r.ultima_pontuacao
        except _CartolaErr as e:
            print(f"⚠️ Erro ao acessar rodada {rodada} (time {time_id}): {e}")
            pontuacoes[rodada] = None
    return pontuacoes

def gerar_df_pontuacoes(api, ids_times):
    try:
        rodada_atual = api.mercado().rodada_atual
    except _CartolaErr:
        # fallback seguro se a API falhar
        rodada_atual = TURNO_INICIO

    colunas_turno = [f"Rodada {i}" for i in range(TURNO_INICIO, TOTAL_RODADAS + 1)]

    # Caso o 2º turno ainda não tenha começado (sem pontuações na R20)
    if not campeonato_comecou(api, ids_times):
        print("📌 2º Turno ainda não começou. Criando estrutura com placeholders (0).")
        df = pd.DataFrame(0, index=list(ids_times.keys()), columns=colunas_turno)
        return df

    # Há ao menos uma pontuação na R20 → montar com rodadas concluídas
    dados = {}
    for nome, time_id in ids_times.items():
        pontuacoes = obter_pontuacao_por_rodada(api, time_id, rodada_atual)
        # Converte para Series com índice "Rodada N"
        s = pd.Series({f"Rodada {k}": v for k, v in pontuacoes.items()})
        dados[nome] = s

    # DataFrame com as rodadas já buscadas (pode estar vazio se rodada_atual==20)
    df = pd.DataFrame.from_dict(dados, orient="index")

    # Garante TODAS as colunas do 2º turno, preenchendo faltantes com 0
    df = df.reindex(columns=colunas_turno, fill_value=0)

    # Calcula "Lider_Rodada" somente se houver pelo menos uma rodada com algum valor não-nulo
    colunas_com_dados = [c for c in colunas_turno if df[c].notna().any()]
    if len(colunas_com_dados) > 0 and df[colunas_com_dados].astype(float).sum().sum() != 0:
        df.loc['Lider_Rodada'] = df[colunas_com_dados].idxmax(axis=0)
    # Caso contrário, não cria a linha de líder (evita líder "aleatório" com tudo 0)

    return df

# === Exemplo de uso ===
# ids_times = {v: k for k, v in nomes_por_id.items()}  # índice = nome do time
df_pontuacoes = gerar_df_pontuacoes(api, ids_times)

# Visualização como você já faz
display(df_pontuacoes.T)

""


In [9]:
# def campeonato_comecou(api, ids_times):
#     """Verifica se o campeonato já começou observando a pontuação na 1ª rodada."""
#     for time_id in ids_times.values():
#         try:
#             pontuacao = api.time(time_id=time_id, rodada=20).ultima_pontuacao
#             if pontuacao is not None:
#                 return True
#         except cartolafc.errors.CartolaFCError:
#             continue
#     return False

# def obter_pontuacao_por_rodada(api, time_id, rodada_atual):
#     """Obtém a pontuação do time em cada rodada até a rodada atual."""
#     pontuacoes = {}
#     for rodada in range(20, rodada_atual):
#         try:
#             time_rodada = api.time(time_id=time_id, rodada=rodada)
#             pontuacoes[rodada] = time_rodada.ultima_pontuacao
#         except cartolafc.errors.CartolaFCError as e:
#             print(f"Erro ao acessar pontuação da rodada {rodada} para o time {time_id}: {e}")
#             pontuacoes[rodada] = None
#     return pontuacoes

# def gerar_df_pontuacoes(api, ids_times):
#     rodada_atual = api.mercado().rodada_atual
#     total_rodadas = 38

#     if not campeonato_comecou(api, ids_times):
#         print("📌 O campeonato ainda não começou. Criando estrutura com placeholders.")
#         df = pd.DataFrame(index=ids_times.keys(), columns=[f'Rodada {i}' for i in range(20, total_rodadas + 1)])
#         df[:] = 0
#     else:
#         df = pd.DataFrame()
#         for nome, time_id in ids_times.items():
#             pontuacoes = obter_pontuacao_por_rodada(api, time_id, rodada_atual)
#             df[nome] = pd.Series(pontuacoes)
#         df = df.transpose()
#         df.columns = [f'Rodada {i}' for i in range(1, rodada_atual)]
#         df.loc['Lider_Rodada'] = df.idxmax()
    
#     return df

In [10]:
ids_times = {v: k for k, v in nomes_por_id.items()}

df_pontuacoes = gerar_df_pontuacoes(api, ids_times)

display(df_pontuacoes.head(20))

""


### Função para definir a classificação dos times

In [22]:
def classificacao_por_grupo(df_rodadas: pd.DataFrame, df_pontuacoes: pd.DataFrame):
    """
    Classificação dos grupos com base nos confrontos e nas pontuações do Cartola.

    Mesmo sem pontuações (rodada não finalizada), retorna uma tabela
    com todos os times dos confrontos zerados.
    """
    # tira marcador se existir
    df_pontuacoes_times = df_pontuacoes.drop(index='Lider_Rodada', errors='ignore')

    # 1) Pré-inicializa todos os times (zerados) por grupo, a partir dos confrontos
    estatisticas = {}
    for _, r in df_rodadas.iterrows():
        grupo = r["Grupo"]
        mandante = r["Mandante_Nome"]
        visitante = r["Visitante_Nome"]
        if grupo not in estatisticas:
            estatisticas[grupo] = {}
        for time in (mandante, visitante):
            if time not in estatisticas[grupo]:
                estatisticas[grupo][time] = {
                    "Pontos": 0,
                    "Vitórias": 0,
                    "Empates": 0,
                    "Derrotas": 0,
                    "Total_Cartola": 0.0,
                    "Cartola_Sofrido": 0.0,
                }

    # helper seguro para ler pontuação; None => não considerar o jogo
    def safe_pontuacao(nome_time: str, coluna_rodada: str):
        if (nome_time in df_pontuacoes_times.index) and (coluna_rodada in df_pontuacoes_times.columns):
            val = df_pontuacoes_times.at[nome_time, coluna_rodada]
            if pd.notnull(val):
                try:
                    return float(val)
                except Exception:
                    return None
        return None

    # 2) Processa os confrontos apenas quando existir pontuação válida pros 2 times
    for _, confronto in df_rodadas.iterrows():
        rodada = confronto["Rodada"]
        mandante = confronto["Mandante_Nome"]
        visitante = confronto["Visitante_Nome"]
        grupo = confronto["Grupo"]
        coluna_rodada = f"Rodada {rodada}"

        pm = safe_pontuacao(mandante, coluna_rodada)
        pv = safe_pontuacao(visitante, coluna_rodada)

        # sem dados -> ignora (mantém tudo zerado, não conta jogo)
        if pm is None or pv is None:
            continue

        # totais/cartola
        estatisticas[grupo][mandante]["Total_Cartola"] += pm
        estatisticas[grupo][mandante]["Cartola_Sofrido"] += pv
        estatisticas[grupo][visitante]["Total_Cartola"] += pv
        estatisticas[grupo][visitante]["Cartola_Sofrido"] += pm

        # pontos
        if pm > pv:
            estatisticas[grupo][mandante]["Pontos"] += 3
            estatisticas[grupo][mandante]["Vitórias"] += 1
            estatisticas[grupo][visitante]["Derrotas"] += 1
        elif pm < pv:
            estatisticas[grupo][visitante]["Pontos"] += 3
            estatisticas[grupo][visitante]["Vitórias"] += 1
            estatisticas[grupo][mandante]["Derrotas"] += 1
        else:
            estatisticas[grupo][mandante]["Pontos"] += 1
            estatisticas[grupo][visitante]["Pontos"] += 1
            estatisticas[grupo][mandante]["Empates"] += 1
            estatisticas[grupo][visitante]["Empates"] += 1

    # 3) Monta o DataFrame final (mesmo que tudo esteja 0)
    frames = []
    for grupo, times in estatisticas.items():
        if not times:
            continue
        df_g = pd.DataFrame({
            "Grupo": grupo,
            "Nome do Time": list(times.keys()),
            "Pontos": [stats["Pontos"] for stats in times.values()],
            "Vitórias": [stats["Vitórias"] for stats in times.values()],
            "Empates": [stats["Empates"] for stats in times.values()],
            "Derrotas": [stats["Derrotas"] for stats in times.values()],
            "Total Cartola": [stats["Total_Cartola"] for stats in times.values()],
            "Cartola Sofrido": [stats["Cartola_Sofrido"] for stats in times.values()],
        })
        df_g["Saldo Cartola"] = df_g["Total Cartola"] - df_g["Cartola Sofrido"]
        frames.append(df_g)

    if frames:
        df_resultado = pd.concat(frames, ignore_index=True)
    else:
        # Sem confrontos? devolve estrutura vazia com colunas esperadas
        cols = ["Grupo","Nome do Time","Pontos","Vitórias","Empates","Derrotas",
                "Total Cartola","Cartola Sofrido","Saldo Cartola","Posição"]
        return pd.DataFrame(columns=cols), {}

    # ordenação (desempate por Total/Saldo e depois Nome)
    df_resultado = df_resultado.sort_values(
        by=["Grupo", "Pontos", "Total Cartola", "Saldo Cartola", "Nome do Time"],
        ascending=[True, False, False, False, True]
    ).reset_index(drop=True)

    df_resultado["Posição"] = df_resultado.groupby("Grupo").cumcount() + 1

    df_resultado_por_grupo = {
        g: df_resultado[df_resultado["Grupo"] == g].copy()
        for g in df_resultado["Grupo"].unique()
    }

    return df_resultado, df_resultado_por_grupo

In [23]:
# def classificacao_por_grupo(df_rodadas, df_pontuacoes):
#     """
#     Classificação dos grupos com base nos confrontos e nas pontuações do Cartola.

#     Retorna:
#     - df_resultado: classificação geral
#     - df_resultado_por_grupo: dicionário com classificação separada por grupo
#     """
#     df_pontuacoes_times = df_pontuacoes.drop(index='Lider_Rodada', errors='ignore')
#     estatisticas = {}

#     for _, confronto in df_rodadas.iterrows():
#         rodada = confronto["Rodada"]
#         mandante = confronto["Mandante_Nome"]
#         visitante = confronto["Visitante_Nome"]
#         grupo = confronto["Grupo"]
#         coluna_rodada = f"Rodada {rodada}"

#         if mandante not in df_pontuacoes_times.index or visitante not in df_pontuacoes_times.index:
#             continue
#         if coluna_rodada not in df_pontuacoes_times.columns:
#             continue

#         pontos_mandante = df_pontuacoes_times.at[mandante, coluna_rodada]
#         pontos_visitante = df_pontuacoes_times.at[visitante, coluna_rodada]

#         if pd.isnull(pontos_mandante) or pd.isnull(pontos_visitante):
#             continue

#         for time in [mandante, visitante]:
#             if grupo not in estatisticas:
#                 estatisticas[grupo] = {}
#             if time not in estatisticas[grupo]:
#                 estatisticas[grupo][time] = {
#                     "Pontos": 0, "Vitórias": 0, "Empates": 0, "Derrotas": 0,
#                     "Total_Cartola": 0, "Cartola_Sofrido": 0
#                 }

#         # Atualizar estatísticas do jogo
#         estatisticas[grupo][mandante]["Total_Cartola"] += pontos_mandante
#         estatisticas[grupo][mandante]["Cartola_Sofrido"] += pontos_visitante

#         estatisticas[grupo][visitante]["Total_Cartola"] += pontos_visitante
#         estatisticas[grupo][visitante]["Cartola_Sofrido"] += pontos_mandante

#         if pontos_mandante > pontos_visitante:
#             estatisticas[grupo][mandante]["Pontos"] += 3
#             estatisticas[grupo][mandante]["Vitórias"] += 1
#             estatisticas[grupo][visitante]["Derrotas"] += 1
#         elif pontos_mandante < pontos_visitante:
#             estatisticas[grupo][visitante]["Pontos"] += 3
#             estatisticas[grupo][visitante]["Vitórias"] += 1
#             estatisticas[grupo][mandante]["Derrotas"] += 1
#         else:
#             estatisticas[grupo][mandante]["Pontos"] += 1
#             estatisticas[grupo][visitante]["Pontos"] += 1
#             estatisticas[grupo][mandante]["Empates"] += 1
#             estatisticas[grupo][visitante]["Empates"] += 1

#     # Montar DataFrame final
#     df_resultado = pd.concat([
#         pd.DataFrame({
#             "Grupo": grupo,
#             "Nome do Time": list(times.keys()),
#             "Pontos": [stats["Pontos"] for stats in times.values()],
#             "Vitórias": [stats["Vitórias"] for stats in times.values()],
#             "Empates": [stats["Empates"] for stats in times.values()],
#             "Derrotas": [stats["Derrotas"] for stats in times.values()],
#             "Total Cartola": [stats["Total_Cartola"] for stats in times.values()],
#             "Cartola Sofrido": [stats["Cartola_Sofrido"] for stats in times.values()],
#             "Saldo Cartola": [stats["Total_Cartola"] - stats["Cartola_Sofrido"] for stats in times.values()
#             ]
#         })
#         for grupo, times in estatisticas.items()
#     ], ignore_index=True)


#     df_resultado = df_resultado.sort_values(
#         by=["Grupo", "Pontos", "Total Cartola"],
#         ascending=[True, False, False]
#     )

#     df_resultado["Posição"] = df_resultado.groupby("Grupo")\
#     .cumcount() + 1

#     df_resultado_por_grupo = {
#         grupo: df_resultado[df_resultado["Grupo"] == grupo] for grupo in df_resultado["Grupo"].unique()
#     }

#     return df_resultado, df_resultado_por_grupo

In [24]:
# 1. Carregar confrontos
df_confrontos = pd.read_csv("confrontos_serie_A.csv")

# 2. Renomear colunas se necessário
df_confrontos.rename(columns={
    "Time A": "Mandante_Nome",
    "Time B": "Visitante_Nome"
}, inplace=True)

# 3. Adicionar coluna de grupo fixo
df_confrontos["Grupo"] = "Série A"

display(df_confrontos.head())

# 4. Calcular classificação
df_classificacao, _ = classificacao_por_grupo(df_confrontos, df_pontuacoes)

# 5. Ver resultado
display(df_classificacao.head(20))

,Rodada,Confronto,Mandante_Nome,Visitante_Nome,ID A,ID B,Grupo
0,1,1,Rolo Compressor ZN,FBC Colorado,11,6,Série A
1,1,2,BORGES ITAQUI F.C.,Analove10 ITAQUI GRANDE!!,3,2,Série A
2,1,3,cartola scheuer,seralex,17,20,Série A
3,1,4,Texas Club 2025,pura bucha /botafogo,15,19,Série A
4,1,5,lsauer fc,RHANKA DENTY FC25,18,10,Série A


,Grupo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
0,Série A,A Lenda Super Vasco F.c,0,0,0,0,0.00,0.00,0.00,1
1,Série A,Analove10 ITAQUI GRANDE!!,0,0,0,0,0.00,0.00,0.00,2
2,Série A,BORGES ITAQUI F.C.,0,0,0,0,0.00,0.00,0.00,3
3,Série A,DM Studio,0,0,0,0,0.00,0.00,0.00,4
4,Série A,Dom Camillo68,0,0,0,0,0.00,0.00,0.00,5
5,Série A,FBC Colorado,0,0,0,0,0.00,0.00,0.00,6
6,Série A,Fedato Futebol Clube,0,0,0,0,0.00,0.00,0.00,7
7,Série A,Gremiomaniasm,0,0,0,0,0.00,0.00,0.00,8
8,Série A,MauHumor F.C.,0,0,0,0,0.00,0.00,0.00,9
9,Série A,RHANKA DENTY FC25,0,0,0,0,0.00,0.00,0.00,10


In [25]:
# 1. Times únicos nos confrontos
times_confrontos = pd.unique(df_confrontos[["Mandante_Nome", "Visitante_Nome"]].values.ravel())

# 2. Times únicos na classificação final
times_classificados = df_classificacao["Nome do Time"].unique()

# 3. Ver quem está nos confrontos mas não foi classificado
faltando = set(times_confrontos) - set(times_classificados)

print("❌ Times que estão nos confrontos, mas faltam na classificação:")
print(faltando)

# df_pontuacoes.loc[["Time do S.A.P.O", "BORGES ITAQUI F.C."]]


❌ Times que estão nos confrontos, mas faltam na classificação:
set()


In [26]:
# Exportar resultado em CSV
df_classificacao.to_csv("classificacao_serie_a.csv", index=False)

# Carregar o arquivo CSV enviado
df_classificacao = pd.read_csv("classificacao_serie_a.csv")

# Renomear colunas para os nomes usados no JavaScript
df_classificacao.rename(columns={
    "Grupo": "grupo",
    "Nome do Time": "nome",
    "Pontos": "pontos",
    "Vitórias": "vitorias",
    "Empates": "empates",
    "Derrotas": "derrotas",
    "Total Cartola": "totalCartola",
    "Cartola Sofrido": "cartolaSofrido",
    "Saldo Cartola": "saldoCartola",
    "Posição": "posicao"
}, inplace=True)

# Agrupar por grupo (apenas Série A nesse caso)
classificacao_por_grupo = {}
for grupo, dados in df_classificacao.groupby("grupo"):
    classificacao_por_grupo[grupo] = dados.drop(columns="grupo").to_dict(orient="records")

# Salvar como arquivo JavaScript
caminho_saida = "classificacao_serie_A.js"
with open(caminho_saida, "w", encoding="utf-8") as f:
    f.write(f"const classificacaoSerieA = {json.dumps(classificacao_por_grupo, indent=2, ensure_ascii=False)};")

caminho_saida

'classificacao_serie_A.js'

In [27]:
import pandas as pd
import json

# Carregar CSV
df_confrontos = pd.read_csv("confrontos_serie_A.csv")

# Limpar espaços nas colunas
df_confrontos.columns = df_confrontos.columns.str.strip()

# Renomear colunas
df_confrontos.rename(columns={
    "Rodada": "rodada",
    "Confronto": "confronto",
    "Time A": "mandante_nome",
    "Time B": "visitante_nome",
    "ID A": "mandante_id",
    "ID B": "visitante_id"
}, inplace=True)

# Adicionar grupo fixo
df_confrontos["grupo"] = "Série A"

# Gerar estrutura para JS
confrontos_formatado = []
for _, row in df_confrontos.iterrows():
    confronto = {
        "rodada": int(row["rodada"]),
        "confronto": int(row["confronto"]),
        "grupo": row["grupo"],
        "mandante": {"id": row["mandante_id"], "nome": row["mandante_nome"]},
        "visitante": {"id": row["visitante_id"], "nome": row["visitante_nome"]}
    }
    confrontos_formatado.append(confronto)

# Salvar como arquivo JS
js_output = f"const confrontosFase1 = {json.dumps(confrontos_formatado, indent=2, ensure_ascii=False)};"
with open("confrontos_serie_A.js", "w", encoding="utf-8") as f:
    f.write(js_output)

print("✅ Arquivo confrontos_serie_A.js gerado com sucesso!")


✅ Arquivo confrontos_serie_A.js gerado com sucesso!


In [28]:
# Carregar o arquivo CSV enviado
df_confrontos = pd.read_csv("confrontos_serie_A.csv")

display(df_confrontos.head())

,Rodada,Confronto,Time A,Time B,ID A,ID B
0,1,1,Rolo Compressor ZN,FBC Colorado,11,6
1,1,2,BORGES ITAQUI F.C.,Analove10 ITAQUI GRANDE!!,3,2
2,1,3,cartola scheuer,seralex,17,20
3,1,4,Texas Club 2025,pura bucha /botafogo,15,19
4,1,5,lsauer fc,RHANKA DENTY FC25,18,10


In [29]:
def gerar_resultados_serie_a(df_confrontos, df_pontuacoes, grupo="Série A"):
    """
    Gera um DataFrame com os resultados dos confrontos da Série A por rodada.
    """
    resultados = []

    for _, row in df_confrontos.iterrows():
        rodada = row["Rodada"]
        mandante = row["Time A"]
        visitante = row["Time B"]

        pontos_mandante = df_pontuacoes.get(f"Rodada {rodada}", {}).get(mandante, None)
        pontos_visitante = df_pontuacoes.get(f"Rodada {rodada}", {}).get(visitante, None)

        resultados.append({
            "grupo": grupo,
            "rodada": rodada,
            "mandante": {
                "nome": mandante,
                "pontos": pontos_mandante
            },
            "visitante": {
                "nome": visitante,
                "pontos": pontos_visitante
            }
        })

    return pd.DataFrame(resultados)


In [30]:
df_resultados = gerar_resultados_serie_a(df_confrontos, df_pontuacoes)

# Exportar para .js
import json

with open("resultados_serie_A.js", "w", encoding="utf-8") as f:
    f.write("const resultadosFase1 = ")
    f.write(json.dumps(df_resultados.to_dict(orient="records"), indent=2, ensure_ascii=False))
    f.write(";")

print("✅ Arquivo resultados_serie_A.js gerado com sucesso.")


✅ Arquivo resultados_serie_A.js gerado com sucesso.
